In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym

In [8]:
env = gym.make("MountainCar-v0")

In [66]:
from torch import nn
import torch.nn.functional as F
from gymnasium import spaces
import numpy as np
from tqdm import tqdm

class TDLambda(nn.Module):
    def __init__(self, env, lam: float = 0.9, discount: float = 0.95, alpha: float = 0.05, action_size = 3, observation_size = 2):
        self.lam = lam
        self.discount = discount
        self.alpha = alpha

        # environment specific vars
        self.action_space = spaces.Discrete(action_size)
        self.action_size = action_size
        self.observation_size = observation_size
        
        self.env = env

        # metrics 
        self.actions = []
        self.rewards = []
        self.observations = []

        self.E = {}
        self.V = {}

        self.policy = {}

    def init_policy(self):
        for i in np.linspace(-1.2, 0.6, 19, dtype=np.float32):
            for j in np.linspace(-0.07, 0.07, 15, dtype=np.float32):
                self.policy[(i, j)] = list(np.random.random_sample((3,)))

    def train(self, total_steps: int = 5000):

        self.init_policy()
        self.initialize_values_and_traces()
        obs = self.env.reset()
        obs = obs[0]
        self.observations.append(obs)
        self.rewards.append(0)

        action = None

        for step in tqdm(range(total_steps), desc="training td-lambda"):
            action = self.choose_action()
            prevobs = obs
            obs, rew, done, trunc, info = self.env.step(action)
            td = rew + self.discount * self.value(obs) - self.value(prevobs)
            self.eligibility_trace(prevobs)

            for s in self.V.keys():
                self.V[s] = self.V[s] + self.alpha * td * self.E[s]
                self.E[s] = self.discount * self.lam * self.E[s]

            self.policy[(np.round(prevobs[0], 1), np.round(prevobs[1], 2))][action] += self.alpha * (self.rewards[-1] + self.discount * self.value(obs))

            self.observations.append(obs)
            self.rewards.append(rew)
            self.actions.append(action)

            if done or trunc:
                self.reset_traces()
                obs = self.env.reset()
                obs = obs[0]
                self.rewards.append(0)
                self.observations.append(obs)


    def eval(self, episodes: int = 5):
        self.env.render()
        obs = self.env.reset()
        rewards = []
        cur_reward = 0

        for e in range(episodes):
            action = np.argmax()


    def initialize_values_and_traces(self):
        for i in np.linspace(-1.2, 0.6, 19, dtype=np.float32):
            for j in np.linspace(-0.07, 0.07, 15, dtype=np.float32):
                if i>=0.5: 
                    self.V[(i, j)] = 0
                    self.E[(i, j)] = 0
                    continue
                self.V[(i, j)] = np.random.random_sample()
                self.E[(i, j)] = 0
    

    def reset_traces(self):
        for i in np.linspace(-1.2, 0.6, 19, dtype=np.float32):
            for j in np.linspace(-0.07, 0.07, 15, dtype=np.float32):
                if i>=0.5: 
                    self.E[(i, j)] = 0
                    continue
                self.E[(i, j)] = 0


    def eligibility_trace(self, state):
        pos = np.round(state[0], 1)
        vel = np.round(state[1], 2)
        value = (pos, vel)
        self.E[value] += 1
        return self.E[value]


    def value(self, state):
        pos = np.round(state[0], 1)
        vel = np.round(state[1], 2)
        value = (pos, vel)
        return self.V[value]


    def choose_action(self):
        return self.action_space.sample()

In [67]:
model = TDLambda(env)
model.train()

training td-lambda: 100%|██████████| 5000/5000 [00:02<00:00, 1677.28it/s]
